# State and Output Schema

In [38]:
from typing import TypedDict

from langchain_core.pydantic_v1 import BaseModel, Field


# Pydantic model to structure the LLM's output
class ScorePrediction(BaseModel):
    """The structured output for the resume score."""

    score: float = Field(description="The matching score between JD and Resume (0–10)")
    explanation: str = Field(description="The explanation of the matching score")


# The state that will be passed through the graph
class GraphState(TypedDict):
    """The state of our graph."""

    jd: str
    resume: str
    prediction: ScorePrediction  # The final output will be stored here

# Nodes

In [39]:
import re

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

# The complete, optimized prompt generated by DSPy
OPTIMIZED_PROMPT_TEMPLATE = """
# SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text
Your output fields are:
1. `score` (str): The matching score between JD and Resume (0–10)
2. `explanation` (str): The explanation of the matching score
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## jd ## ]]
{{jd}}

[[ ## resume ## ]]
{{resume}}

[[ ## score ## ]]
{{score}}

[[ ## explanation ## ]]
{{explanation}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given a job description and a resume, predict a matching score (0-10).

# FEW-SHOT EXAMPLE 1
[[ ## jd ## ]]
Software Engineer needed with experience in Git, Docker, Java, REST APIs. Hit summer discussion culture measure ever.

[[ ## resume ## ]]
Experienced professional skilled in System Design, past, Java, foot. Thank case rather generation inside. Raise new structure race.

[[ ## score ## ]]
4

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 2
[[ ## jd ## ]]
ML Engineer needed with experience in Python, PyTorch, Computer Vision, TensorFlow, Keras. Could chair beautiful social both few through. Entire card much rate politics their identify. Pass sing goal during be those.

[[ ## resume ## ]]
Experienced professional skilled in Computer Vision, Python, MLOps, PyTorch, TensorFlow, parent. Exactly section network detail. Short out team author deal hospital able.

[[ ## score ## ]]
10

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 3
[[ ## jd ## ]]
ML Engineer needed with experience in TensorFlow, Computer Vision, Keras, PyTorch, MLOps. Approach wish fine near. Agree long behind stuff how positive tree. Quality team general office painting official.

[[ ## resume ## ]]
Experienced professional skilled in Cloud, interest, MLOps, Keras, hotel, PyTorch. Address guy fund window well impact quite. Place raise really feeling vote per. Catch board present market society fight foreign. Out generation beyond six degree stop.

[[ ## score ## ]]
8

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 4
[[ ## jd ## ]]
Data Scientist needed with experience in Deep Learning, Machine Learning, NLP, SQL, Pandas, Python, Statistics. Nearly computer close garden. Law individual business hair show. Final though kitchen purpose five. Use sea right civil.

[[ ## resume ## ]]
Experienced professional skilled in Python, Deep Learning, garden, Statistics, SQL, Pandas, stock. Note operation despite born. Step take share million message long board.

[[ ## score ## ]]
8

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FINAL QUERY
[[ ## jd ## ]]
{jd}

[[ ## resume ## ]]
{resume}

Respond with the corresponding output fields, starting with the field `[[ ## score ## ]]`, then `[[ ## explanation ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.
"""


def scorer_node(state: GraphState):
    """Invokes the LLM to score the resume against the job description."""

    model = ChatOllama(
        model="llama3.2:1b",  # Or another model like 'llama3:70b', 'mistral', etc.
        temperature=0,
    )

    # Create the prompt from the template
    prompt = ChatPromptTemplate.from_template(OPTIMIZED_PROMPT_TEMPLATE)

    # Create the chain
    chain = prompt | model

    # Get inputs from the state
    jd = state["jd"]
    resume = state["resume"]

    # Invoke the chain
    response = chain.invoke({"jd": jd, "resume": resume})
    response_text = response.content

    # Parse the structured output from the model's response
    try:
        score_match = re.search(r"\[\[ ## score ## \]\]\s*([\d.]+)", response_text)
        explanation_match = re.search(
            r"\[\[ ## explanation ## \]\]\s*([\s\S]*?)\s*\[\[ ## completed ## \]\]",
            response_text,
        )

        if not score_match or not explanation_match:
            raise ValueError("Could not find score or explanation tags in the output.")

        score_val = float(score_match.group(1).strip())
        explanation_val = explanation_match.group(1).strip()

        prediction = ScorePrediction(score=score_val, explanation=explanation_val)

    except (ValueError, AttributeError) as e:
        print(f"Error parsing LLM output: {e}\nResponse: {response_text}")
        # Assign a default low score on parsing failure
        prediction = ScorePrediction(
            score=0.0, explanation="Failed to parse model output."
        )

    return {"prediction": prediction}

# Graph

In [40]:
from langgraph.graph import END, StateGraph

# Initialize the graph
workflow = StateGraph(GraphState)

# Add the scorer node
workflow.add_node("scorer", scorer_node)

# Set the entry point and the final edge
workflow.set_entry_point("scorer")
workflow.add_edge("scorer", END)

# Compile the graph into a runnable application
app = workflow.compile()

print("\n✅ LangGraph App Compiled Successfully!")

# --- Example Usage ---
jd_example = "Product Manager needed with experience in User Stories, Product Roadmap, Stakeholder Management, Agile."
resume_example = "Experienced professional skilled in blood, Scrum, skill, Stakeholder Management, quality, Agile."

inputs = {"jd": jd_example, "resume": resume_example}

# Run the graph
result = app.invoke(inputs)

print("\n--- Example Result ---")
print(f"Score: {result['prediction'].score}")
print(f"Explanation: {result['prediction'].explanation}")


✅ LangGraph App Compiled Successfully!


/var/folders/pk/36_zplxd57l142tqyrqqmmkh0000gp/T/ipykernel_91454/4204216905.py:107: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(



--- Example Result ---
Score: 4.0
Explanation: Not supplied for this particular example.


# Metrics

In [41]:
import numpy as np
from sklearn.metrics import mean_absolute_error


def accuracy_at_threshold(gold_scores, pred_scores, threshold):
    gold_scores = np.array(gold_scores)
    pred_scores = np.array(pred_scores)
    absolute_errors = np.abs(gold_scores - pred_scores)
    within_threshold_count = np.sum(absolute_errors <= threshold)
    return within_threshold_count / len(gold_scores)


# Evaluate

## Kaggle dataset

In [43]:
import pandas as pd

train_df = pd.read_csv("../../data/train.csv")
val_df = pd.read_csv("../../data/test.csv")

In [44]:
import time

from tqdm import tqdm

gold_scores = []
pred_scores = []

print(f"\n--- Starting Evaluation on {len(val_df)} examples ---")

for i, example in tqdm(
    val_df.iterrows(), desc="Evaluating LangGraph", total=len(val_df)
):
    # Get ground truth data
    jd = example["job_description"]
    resume = example["resume"]
    gold_score = example["match_score"]

    # Run the LangGraph app
    inputs = {"jd": jd, "resume": resume}
    try:
        result = app.invoke(inputs)
        pred_score = result["prediction"].score
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(1)

print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")


--- Starting Evaluation on 3000 examples ---


Evaluating LangGraph:   0%|          | 8/3000 [02:46<16:40:15, 20.06s/it]

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: # SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text

## JD
Software Engineer needed with experience in Git, Java, System Design, REST APIs, Docker.

## Resume
Experienced professional skilled in System Design, cultural, race, Docker, Java, Spring Boot, Git. Again race agency late energy avoid. Instead get property instead gun decide laugh. Number beat none mention agree room.

## Score
8

## Explanation
The matching score between JD and Resume is 8. This suggests that the candidate's resume contains relevant information about their experience in software engineering, system design, and Docker, but does not explicitly state this. The explanation for the match is "Not supplied".

## Completed


Evaluating LangGraph:   1%|          | 31/3000 [2:34:14<116:50:13, 141.67s/it] 

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: # SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text

## JD
Software Engineer needed with experience in REST APIs, Docker, Java, Spring Boot, Git.

## Resume
Experienced professional skilled in Git, Docker, much, Spring Boot, Agile. Good continue tonight apply.

## Score
8

## Explanation
The matching score between the job description and the resume is 8 because both mention relevant skills such as REST APIs, Docker, Java, and Spring Boot.

## Completed


...

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: # SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text

## JD
Software Engineer needed with experience in Java, Docker, System Design, Spring Boot, Agile, REST APIs, Git.

## Resume
Experienced professional skilled in body, Java, System Design, Docker, Agile. Memory single modern itself other. Cover hospital end. Five wish stuff only. Institution information bill collection tend this.

## Score
8

## Explanation
The matching score between JD and Resume is 8 because the candidate's resume mentions experience with Spring Boot, which is not mentioned in the job description. The explanation for the mismatch is that the job description does not mention Docker or Agile, but the resume mentions these technologies.

## Completed


Evaluating LangGraph:  15%|█▌        | 455/3000 [18:24:00<80:46:33, 114.26s/it] 

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: # SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text

## JD
Software Engineer needed with experience in System Design, Docker, Spring Boot, Git, Java, REST APIs, Agile.

## Resume
Experienced professional skilled in Java, interview, Docker. Model although national anything fine low make. Give voice only visit.

## Score
8

## Explanation
The matching score between the job description and resume is 8 out of 10. The candidate's experience in System Design, Spring Boot, Git, and Java aligns well with the job requirements, while their experience in Docker and Agile also shows a good fit. However, there are no specific skills mentioned that match the job description perfectly.

## Completed
No


Evaluating LangGraph:  23%|██▎       | 703/3000 [31:14:52<102:06:01, 160.02s/it]


KeyboardInterrupt: 

## Collected dataset

In [47]:
import json
import time
import os
from tqdm import tqdm

collected_folder = "/Users/nhannguyen/ngtuthanhan@gmail.com - Google Drive/My Drive/HCMUS/LLM/Data_Collector_30_8"
matching_json = os.path.join(collected_folder, "Matching_Result.json")

with open(matching_json, "r") as f:
    matching_data = json.load(f)

pred_scores = []
gold_scores = []

for example in tqdm(
    matching_data, desc="Evaluating LangGraph", total=len(matching_data)
):
    jd_path = example["JD"]
    resume_path = example["CV"]
    gold_score = example["Score"]
    with open(os.path.join(collected_folder, "JD", jd_path) + ".txt", "r") as f:
        jd = f.read()
    with open(os.path.join(collected_folder, "CV", resume_path) + ".txt", "r") as f:
        resume = f.read()
    inputs = {"jd": jd, "resume": resume}
    try:
        result = app.invoke(inputs)
        pred_score = result["prediction"].score
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(1)


print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")

Evaluating LangGraph:  11%|█▏        | 10/87 [02:12<13:44, 10.71s/it]

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
{score}

[[ ## explanation ## ]]
The matching score between JD and Resume is 8. This score indicates that the job description provided in the job posting matches the skills and experience required by the candidate's resume.

[[ ## completed ## ]]

Note: The output fields are filled with the corresponding values based on the analysis of the input fields "jd" and "resume".


Evaluating LangGraph:  14%|█▍        | 12/87 [02:26<11:23,  9.11s/it]

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
{8}

[[ ## explanation ## ]]
The matching score between JD and Resume is 8. This indicates that there are some similarities in the job description and resume, but not enough to guarantee a perfect match.

The key areas where the two documents show similarities include:

* The use of technical terms related to data science and machine learning
* The mention of specific tools and technologies such as Hadoop, Spark, and PySpark
* The emphasis on data preprocessing, feature engineering, and model training

However, there are also some differences that prevent a perfect match. For example:

* The job description mentions the need for "quick access to legal information" which is not explicitly mentioned in the resume
* The resume does not mention any experience with Power BI or Hugging Face models
* There is no mention of specific data visualization tools such as Tableau or D3.js

Ov

Evaluating LangGraph:  15%|█▍        | 13/87 [02:40<12:42, 10.30s/it]

...

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: Here is the response with the corresponding output fields:

```markdown
# SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text
Your output fields are:
1. `score` (str): The matching score between JD and Resume (0–10)
2. `explanation` (str): The explanation of the matching score
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## jd ## ]]
{jd}

[[ ## resume ## ]]
{resume}

[[ ## score ## ]]
{score}

[[ ## explanation ## ]]
{explanation}

[[ ## completed ## ]]
```

Let me know if you'd like me to fill in any specific details or examples.


Evaluating LangGraph:  99%|█████████▉| 86/87 [19:54<00:15, 15.16s/it]

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
8

[[ ## explanation ## ]]
The matching score between JD and Resume is based on the similarity in job description, skills, and experience. The candidate's resume highlights their relevant experience as a Full Stack Engineer, while the job description emphasizes the need for an experienced engineer with expertise in web development, APIs, and software testing.

The key similarities are:

* Experience in building modern web applications
* Proficiency in Java/Kotlin/Go/Rust/Python programming languages
* Strong understanding of computer science fundamentals, data structures, and algorithms

However, there are also some differences that might affect the matching score:

* The job description mentions a "summer discussion culture" which is not explicitly mentioned in the candidate's resume.
* The candidate's experience with cloud-based technologies (e.g., AWS/GCP) is not highlighted

Evaluating LangGraph: 100%|██████████| 87/87 [20:02<00:00, 13.82s/it]

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 4.3793
---
🎯 Accuracy@1 (error <= 1.0): 26.44%
🎯 Accuracy@2 (error <= 2.0): 34.48%
🎯 Accuracy@3 (error <= 3.0): 40.23%
--------------------------------------
